In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from mistralai.client import MistralClient
from mistralai import Mistral
from langchain_mistralai import ChatMistralAI

from dataclasses import dataclass

from langchain.chat_models.base import SimpleChatModel
import os
from langchain.chains import RetrievalQA
import streamlit as st




In [2]:
@dataclass
class ChatMessage:
    role: str
    content: str



# Exemple : tu dois utiliser le même modèle d'embedding que celui utilisé pour indexer
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Connexion à ta base Chroma
vectordb = Chroma(persist_directory="chromadb", embedding_function=embedding_model)


retriever = vectordb.as_retriever(search_kwargs={"k": 5})





C:\Users\doria\AppData\Local\Temp\ipykernel_9104\1844848340.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
C:\Users\doria\AppData\Local\Temp\ipykernel_9104\1844848340.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="chromadb", 

In [6]:
os.environ["MISTRAL_API_KEY"] = "1ROdIMBXIGXrgxnD3cywOg4EhRaJufJA"

client = Mistral(api_key="1ROdIMBXIGXrgxnD3cywOg4EhRaJufJA")

mistral_model = "mistral-large-latest" # "open-mixtral-8x22b" 
llm = ChatMistralAI(model=mistral_model, temperature=0.2)


# Chaîne RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # Pour récupérer les chunks utilisés
)

# Exemple de question
question = "Combien de taxi en France ?"
result = qa_chain(question)

print("Réponse :\n", result['result'])

# Pour afficher les sources :
for doc in result["source_documents"]:
    print("\n--- Source ---")
    print(doc.metadata["lien_pdf"])  # ou autre champ
    print(doc.page_content)


Réponse :
 Je ne sais pas. Le texte fourni ne donne pas de chiffre précis sur le nombre total de taxis en France.

--- Source ---
https://www.autoritedelaconcurrence.fr/sites/default/files/commitments//15a20.pdf
434 chauffeurs de taxi contre 23 082 chauffeurs de vtc à paris on dénombre environ 20 000 chauffeurs de taxi contre 3 600 chauffeurs de vtc en angleterre qui ne contingente ni les licences pour taxi ni les licences pour vtc on dénombre par comparaison 75 082 taxis contre 167 118 vtc et à londres 25 200 taxis contre 78 700 vtc 4 16 à londres il est aisé de trouver un taxi dans la rue selon le député thomas thévenoud dans son rapport un taxi pour l’avenir des emplois sur la france la raison serait qu’à londres le marché de la maraude est suffisamment rémunérateur pour que les taxis investissent peu le marché de la réservation préalable le rapport ne précise pourtant pas la raison exacte du caractère plus rémunérateur de la maraude au royaume uni alors que les mêmes règles de tari

2025-06-12 19:35:33.829 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 19:35:35.191 
  command:

    streamlit run C:\Users\doria\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-12 19:35:35.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
